<a href="https://colab.research.google.com/github/macsrc/3d-photo-inpainting/blob/master/fake_news_detection_model_using_tensorflow_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv("news.csv")
data.head()

In [ ]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [ ]:
embedding_dim = 50
max_length = 54
padding_type = 'post'
trunc_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = 0.1

In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(sequences1, padding=padding_type, truncating=trunc_type)

In [ ]:
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [ ]:
training_sequences1 = np.array(training_sequences1)
test_sequences1 = np.array(test_sequences1)

In [ ]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
embedding_index = {}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size1 + 1, embedding_dim))

for word, i in word_index1.items():
    if i < vocab_size1:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1 + 1, embedding_dim, input_length=max_length,
                              weights=[embedding_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    training_sequences1,
    np.array(training_labels),
    epochs=50,
    validation_data=(test_sequences1, np.array(test_labels)),
    verbose=2
)

In [ ]:
X = "Karry to go to France in gesture of sympathy"

sequences = tokenizer1.texts_to_sequences([X])
sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
if model.predict(sequences, verbose=0)[0][0] >= 0.5:
    print("This news is True")
else:
    print("This news is False")

====================================================================================================
# What steps should be taken while working on 'Fake News Detection Model using TensorFlow in Python' program.
In line with AI template, I would like to know step by step process, libraries and key considerations.
----------------------------

Here is a **structured enterprise-style AI template** for:

# 📌 Fake News Detection Model (TensorFlow – Python)

---

# 1️⃣ Business Objective

Detect whether a news article is **Fake or Real** to reduce misinformation spread.

### Success Metrics

* Accuracy ≥ 90%
* High Precision (avoid false accusations)
* Low False Negative Rate
* F1-score

---

# 2️⃣ Problem Definition

* Type: **Binary Text Classification**
* Input: News headline / full article
* Output: Fake (1) / Real (0)

---

# 3️⃣ Data Understanding

### Data Sources

* Kaggle Fake News Dataset
* LIAR dataset
* Custom scraped data

### Key Columns

* title
* text
* label

### Key Considerations

* Class imbalance
* Duplicate news
* Very long articles
* Noise / HTML tags

---

# 4️⃣ Data Preprocessing

### Steps

1. Lowercasing
2. Remove punctuation / special chars
3. Remove stopwords
4. Tokenization
5. Padding sequences
6. Train-test split

### Libraries

* pandas
* numpy
* re
* nltk / spacy
* sklearn (train_test_split)
* tensorflow.keras.preprocessing

---

# 5️⃣ Feature Engineering

### Option A (Basic)

* Tokenizer + Embedding layer

### Option B (Advanced)

* Pretrained embeddings (GloVe)
* BERT embeddings (transformers)

---

# 6️⃣ Modeling Strategy (TensorFlow)

### Baseline Model

* Embedding
* GlobalAveragePooling
* Dense layer
* Sigmoid output

### Advanced Models

* LSTM / BiLSTM
* GRU
* CNN for text
* BERT (via HuggingFace + TF)

### Libraries

* tensorflow / keras
* transformers (optional)

---

# 7️⃣ Model Training

### Loss Function

* BinaryCrossentropy

### Optimizer

* Adam

### Metrics

* Accuracy
* Precision
* Recall
* AUC

### Key Considerations

* Overfitting (use Dropout)
* EarlyStopping
* Class weights (if imbalanced)

---

# 8️⃣ Evaluation Framework

* Confusion Matrix
* Precision-Recall Curve
* ROC Curve
* F1-score
* Cross-validation (optional)

### Libraries

* sklearn.metrics
* matplotlib

---

# 9️⃣ Deployment Architecture

### Option 1

* Save model (.h5 / SavedModel)

### Option 2

* FastAPI REST API

### Option 3

* Streamlit Web App

---

# 🔟 Monitoring & Governance

* Bias detection (political bias)
* Model drift monitoring
* Retraining schedule
* Explainability (LIME / SHAP)

---

# 1️⃣1️⃣ Risks & Key Considerations

* Ethical implications (false labeling)
* Political sensitivity
* Adversarial attacks
* Data freshness
* Hallucination (if using LLM)

---

# 📦 Complete Library Stack

Core:

* pandas
* numpy
* tensorflow / keras
* scikit-learn

NLP:

* nltk / spacy
* transformers (optional)

Deployment:

* fastapi / flask
* streamlit

Visualization:

* matplotlib
* seaborn

---

# 🧠 Enterprise Workflow Summary

1. Business Understanding
2. Data Collection
3. Data Cleaning
4. Feature Engineering
5. Model Development
6. Model Evaluation
7. Deployment
8. Monitoring & Governance

---

If you want, I can:

* ✅ Provide full end-to-end TensorFlow code
* ✅ Provide beginner version + advanced version comparison
* ✅ Convert this into resume-ready enterprise project description
* ✅ Provide architecture diagram explanation (interview ready)

Tell me your goal (learning / interview / Kaggle / portfolio).


================================================================================
# Beginner version code
Here is a **beginner-friendly version** of both projects with simple, clean code.

---

# 🟢 1️⃣ Beginner Fraud Detection (Binary Classification)

Using a small CSV dataset (`fraud.csv`) with:

* `amount`
* `transaction_type`
* `old_balance`
* `new_balance`
* `is_fraud` (0/1)

---

### ✅ Simple Python Code

```python
# Step 1: Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Step 2: Load dataset
data = pd.read_csv("fraud.csv")

# Step 3: Convert categorical column
le = LabelEncoder()
data["transaction_type"] = le.fit_transform(data["transaction_type"])

# Step 4: Define features and target
X = data.drop("is_fraud", axis=1)
y = data["is_fraud"]

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 6: Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 7: Predict
y_pred = model.predict(X_test)

# Step 8: Evaluate
print(classification_report(y_test, y_pred))
```

👉 What you learn:

* Data loading
* Encoding
* Train/test split
* Basic model training
* Evaluation

---

# 🟢 2️⃣ Beginner RAG-style Simple Q&A (Very Basic Version)

This is NOT full enterprise RAG — just a simple concept version.

---

### ✅ Simple Retrieval + GPT-like Response (Mock Example)

```python
# Step 1: Simple knowledge base
documents = {
    "leave policy": "Employees are entitled to 20 days of paid leave.",
    "work hours": "Office hours are 9 AM to 6 PM.",
    "insurance": "Company provides health insurance coverage."
}

# Step 2: Simple search function
def retrieve_answer(query):
    for key in documents:
        if key in query.lower():
            return documents[key]
    return "Sorry, I don't know the answer."

# Step 3: User input
query = input("Ask your question: ")

# Step 4: Get response
response = retrieve_answer(query)

print("Answer:", response)
```

👉 What you learn:

* Basic retrieval concept
* Query matching
* How RAG works conceptually

---

# 🔎 If You Want Slightly More Advanced Beginner Version:

I can provide:

* Fraud detection with **Random Forest**
* Simple RAG using **sentence-transformers**
* Full Jupyter Notebook format
* With sample dataset generation

Tell me your learning goal (Interview / Kaggle / Enterprise understanding).


==============================================================================================
# Provide full end-to-end TensorFlow code

Below is a **complete end-to-end TensorFlow (Keras) example**
👉 Binary Classification – *Credit Card Fraud Detection style* (structured tabular data)

This includes:

* Data loading
* Preprocessing
* Handling imbalance
* Model training
* Evaluation
* Saving model

---

# ✅ End-to-End TensorFlow Code

```python
# ==========================================
# 1. Import Libraries
# ==========================================
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils import class_weight

# ==========================================
# 2. Load Dataset
# Replace with your dataset path
# ==========================================
df = pd.read_csv("creditcard.csv")

print("Shape:", df.shape)
print(df["Class"].value_counts())

# ==========================================
# 3. Split Features & Target
# ==========================================
X = df.drop("Class", axis=1)
y = df["Class"]

# ==========================================
# 4. Train-Test Split
# ==========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ==========================================
# 5. Feature Scaling
# ==========================================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ==========================================
# 6. Handle Class Imbalance
# ==========================================
weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = {0: weights[0], 1: weights[1]}
print("Class Weights:", class_weights)

# ==========================================
# 7. Build Neural Network Model
# ==========================================
model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(32, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", keras.metrics.AUC(name="auc")]
)

model.summary()

# ==========================================
# 8. Train Model
# ==========================================
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=256,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# ==========================================
# 9. Evaluate Model
# ==========================================
loss, accuracy, auc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")

# Predictions
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))

# ==========================================
# 10. Save Model
# ==========================================
model.save("fraud_detection_model.h5")
print("Model saved successfully!")
```

---

# 📌 What This Covers (Enterprise Flow)

✔ Data ingestion
✔ Preprocessing
✔ Imbalance handling
✔ Deep learning model
✔ Evaluation with AUC
✔ Early stopping
✔ Model persistence

---

If you want next level:

* 🔹 Add SHAP explainability
* 🔹 Convert into FastAPI deployment
* 🔹 Add TensorFlow Decision Forest comparison
* 🔹 Convert into Jupyter notebook format
* 🔹 Add model monitoring logic

Tell me your target (learning / interview / enterprise project).


====================================================================================================
# Advanced version of code
-----------------------------------

Below is an **advanced enterprise-style implementation outline (production-ready structure)**.

I’m giving two examples:

1️⃣ **Fraud Detection – Production ML Pipeline**
2️⃣ **RAG System – Enterprise Implementation**

---

# 🔵 1️⃣ Advanced Fraud Detection (Production-Ready)

### 🏗 Architecture Features

* Data validation (Great Expectations style)
* Imbalance handling (SMOTE)
* Pipeline + ColumnTransformer
* Hyperparameter tuning
* Model explainability (SHAP)
* Model versioning ready
* API scoring endpoint

---

## 📌 Project Structure

```
fraud_detection/
│
├── data/
├── models/
├── src/
│   ├── train.py
│   ├── predict.py
│   ├── pipeline.py
│   └── config.py
├── app.py
└── requirements.txt
```

---

## 🔹 pipeline.py

```python
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

def build_pipeline(numeric_features, categorical_features):

    numeric_transformer = Pipeline(steps=[
        ("scaler", StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features)
        ]
    )

    model = RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        random_state=42,
        class_weight="balanced"
    )

    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", model)
    ])

    return pipeline
```

---

## 🔹 train.py

```python
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from pipeline import build_pipeline

# Load data
df = pd.read_csv("data/transactions.csv")

X = df.drop("is_fraud", axis=1)
y = df["is_fraud"]

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

pipeline = build_pipeline(numeric_features, categorical_features)

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [8, 10, 12]
}

grid = GridSearchCV(pipeline, param_grid, cv=5, scoring="roc_auc")
grid.fit(X, y)

print("Best Params:", grid.best_params_)
print("ROC-AUC:", roc_auc_score(y, grid.predict_proba(X)[:,1]))

joblib.dump(grid.best_estimator_, "models/fraud_model.pkl")
```

---

## 🔹 FastAPI Deployment (app.py)

```python
from fastapi import FastAPI
import joblib
import pandas as pd

app = FastAPI()
model = joblib.load("models/fraud_model.pkl")

@app.post("/predict")
def predict(data: dict):
    df = pd.DataFrame([data])
    prediction = model.predict_proba(df)[0][1]
    return {"fraud_probability": float(prediction)}
```

---

## 🔹 Enterprise Add-ons

* MLflow for tracking
* Dockerfile for containerization
* CI/CD pipeline
* Drift detection (Evidently AI)
* SHAP explainability dashboard

---

# 🔵 2️⃣ Advanced RAG System (Enterprise Version)

### 🏗 Stack

* LangChain
* OpenAI / Llama
* FAISS Vector DB
* FastAPI backend
* Role-based access

---

## 🔹 ingestion.py

```python
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

def ingest_documents(file_path):

    loader = PyPDFLoader(file_path)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )

    chunks = splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(chunks, embeddings)

    vectorstore.save_local("vector_db")
```

---

## 🔹 rag_app.py

```python
from fastapi import FastAPI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

app = FastAPI()

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("vector_db", embeddings)
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(model="gpt-4")

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

@app.post("/ask")
def ask_question(query: str):
    response = qa.run(query)
    return {"answer": response}
```

---

## 🔹 Enterprise Enhancements

* RBAC (JWT Auth)
* Prompt guardrails
* Response logging
* Hallucination detection
* Monitoring dashboard
* Hybrid search (BM25 + embeddings)
* Feedback loop for retraining

---

# 🧠 What Makes This “Enterprise-Level”

✔ Modular architecture
✔ Config-driven
✔ Logging + monitoring
✔ Hyperparameter tuning
✔ Model explainability
✔ Deployment ready
✔ Security layers
✔ Scalable

---

If you want next step, I can provide:

* 🔥 Full production-grade version with MLflow + Docker
* 🔥 Cloud deployment (AWS/Azure) version
* 🔥 Complete GitHub-ready project structure
* 🔥 Advanced GenAI guardrails implementation
* 🔥 MLOps maturity roadmap

Tell me which direction you want to go.


